In [1]:
import tensorflow
import tensorflow.keras as keras
import tensorflow.keras.applications as ka
# from tensorflow.keras.keras_preprocessing.image import ImageDataGenerator
import numpy as np
import sys

import pickle

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=1,1

In [22]:
base_model = ka.ResNet50(
                        include_top=True,
                        weights='imagenet',#,
#                         classes=2,
                        input_shape = (224, 224, 3)
                        )

for layer in base_model.layers:
    layer.trainable = False

# # base_model_add_layers = keras.Model(inputs = base_model.input, outputs = base_model.output)
# # out_1 = keras.layers.GlobalAvgPool2D()(base_model.output)
# # out_1 = keras.layers.GlobalAveragePooling2D()(base_model.output)
# x = keras.layers.Flatten()(base_model.layers[-2].output)
x = keras.layers.Dense(1024, activation='sigmoid')(base_model.layers[-2].output)
# x = keras.layers.Dense(1000, activation='sigmoid')(x)
out_1 = keras.layers.Dense(1000, activation='softmax')(x)

model = keras.Model(inputs = base_model.input, outputs = out_1)#, base_model.output, base_model.layers[-2].output])
model_extra = keras.Model(inputs = model.input, outputs = [model.layers[-1].output, model.layers[-2].output, model.layers[-3].output])
# opt = keras.optimizers.SGD(lr = 0.0001)
opt = keras.optimizers.Adam(learning_rate = 0.01)
# base_model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = 'accuracy')
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = 'accuracy')
model_extra.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = 'accuracy')

# model.summary()
# base_model.summary()

In [23]:
# directory = './ImageNet/organized_validation/'

# data_batches = tf.keras.preprocessing.image_dataset_from_directory(
#     directory,
#     labels="inferred",
#     label_mode="categorical",
# #     class_names=None,
#     color_mode="rgb",
#     batch_size=1,
#     image_size=(224, 224),
#     shuffle=True,
#     seed=13,
# #     validation_split=None,
# #     subset=None,
#     interpolation="bilinear"
# #     follow_links=False,
# )

In [24]:
directory = './ImageNet/organized_validation_resnet/'

resnet_preprocess = keras.applications.resnet.preprocess_input

val_id_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = resnet_preprocess,
                                                          validation_split = 0.25)
# val_gen = keras.preprocessing.image.ImageDataGenerator()
# val_id_gen = tensorflow.keras.preprocessing.image.ImageDataGenerator()
# val_gen = ImageDataGenerator()


train_gen = val_id_gen.flow_from_directory(
    directory, 
    target_size=(224, 224),
    color_mode='rgb', 
    classes=None,
    class_mode='categorical', 
    batch_size=8, 
    shuffle=True, 
    seed=13,
    subset='training',
#     save_to_dir=None, 
#     save_prefix='', 
#     save_format='png', f
#     ollow_links=False,
#     subset=None, 
    interpolation='nearest'
)

val_gen = val_id_gen.flow_from_directory(
    directory, 
    target_size=(224, 224),
    color_mode='rgb', 
    classes=None,
    class_mode='categorical', 
    batch_size=32, 
    shuffle=True, 
    seed=13,
    subset='validation',
#     save_to_dir=None, 
#     save_prefix='', 
#     save_format='png', f
#     ollow_links=False,
#     subset=None, 
    interpolation='nearest'
)

# next(val_gen)

Found 38000 images belonging to 1000 classes.
Found 12000 images belonging to 1000 classes.


In [ ]:
history = model.fit(train_gen, validation_data = val_gen, epochs = 10, shuffle = True)

Epoch 1/10
 396/4750 [=>............................] - ETA: 44:22 - loss: 12.5520 - accuracy: 0.0019

In [ ]:
# x_val = np.zeros((10000, 224, 224, 3))
# y_val = np.zeros((10000, 1000))

# x_encoded = np.zeros((50000, 2048))
# y_encoded = np.zeros((50000, 1000))

In [ ]:
for z in range(1, 6):
    for i in range(10000):
        if i % 1000 == 0:
            print(i)

        x, y = val_gen.next()
        x_val[i, :, :, :] = x
        y_val[i, :] = y

    # x_train, x_val, y_train, y_val = train_test_split(x_val, y_val, train_size = .75, shuffle = True)

    out = model_extra.predict(x_val)
    
#     print(a.shape)
#     print(b.shape)
#     print(c.shape)

    
    pickle.dump( out, open( "imagenet_val_1000_encoding_" + str(z) + ".p", "wb" ) )


#     x_encoded[z:z+10000, :] = c
#     y_encoded[z:z+10000, :] = y_val
    

In [ ]:
print(out_intermediate[2].shape)

In [ ]:
out = out_intermediate

dist_from_first = []
image_dict = {}
index = 1444
first_features = out[2][index]
first_image = x_val[index][:,:,:]
print(first_image.shape)

# hash images:
for idx in range(x_val.shape[0]):
    image_dict[str(out[2][idx])] = x_val[idx] 

plt.clf()
plt.imshow(first_image/255)
plt.show()

# for i in range(400):
#     if i % 25 == 0:
#         print(i)
for idx, item in enumerate(out[2]):
    dist_from_first.append((np.linalg.norm(first_features-item), (str(item))))

dist_from_first.sort()
# print(dist_from_first)

for i in range(20):
    plt.clf()
    img = image_dict[dist_from_first[i][1]]
    plt.imshow(img/255)
    plt.show()